In [1]:
import pandas as pd
import matplotlib as plt
%matplotlib inline
import seaborn as sns
import pandas_profiling


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
profile = train.profile_report(title='Haribon Profiling Report')
profile.to_file(output_file="haribon_pandas_profiling.html")

profile = test.profile_report(title='Test Haribon Profiling Report')
profile.to_file(output_file="test_haribon_pandas_profiling.html")

## Data Prep

In [3]:
train.set_index(keys='indexId',inplace=True)
train.head()

,Scientific Name,Annual Mean Temperature,Mean Diurnal Range,Isothermality,Temperature Seasonality,Max Temperature of Warmest Month,Min Temperature of Coldest Month,Temperature Annual Range,Mean Temperature of Wettest Quarter,Mean Temperature of Driest Quarter,Mean Temperature of Warmest Quarter,Mean Temperature of Coldest Quarter,Annual Precipitation,Precipitation of Wettest Month,Precipitation of Driest Month,Precipitation Seasonality,Precipitation of Wettest Quarter,Precipitation of Driest Quarter,Precipitation of Warmest Quarter,Precipitation of Coldest Quarter
indexId,,,,,,,,,,,,,,,,,,,,
1,Cacatua haematuropygia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anas luzonica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cacatua haematuropygia,63.090,19.902,2432.0,25.639,117.493,420.0,7.148,25.357,462.0,178.0,896.0,11.330,26.975,52.334,31.232,53.0,305.0,26.028,23.993
8,Pithecophaga jeffeyri,66.983,20.849,2044.0,26.936,112.042,166.0,8.192,25.870,419.0,71.0,971.0,12.230,28.357,73.092,33.079,15.0,340.0,27.275,25.442
10,Cacatua haematuropygia,76.739,21.074,2113.0,26.943,63.116,569.0,9.091,27.528,589.0,429.0,663.0,11.847,27.757,26.627,32.921,135.0,298.0,27.095,26.139


In [4]:
X = train.drop('Scientific Name', axis=1)
y = train['Scientific Name']

In [5]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS
X = pd.DataFrame(fast_knn(X.values, k=30), columns = X.columns)

In [6]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

In [7]:
from sklearn import model_selection as ms
X_train,X_test,y_train,y_test = ms.train_test_split(X_res,y_res,test_size=0.2)

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
XG = XGBClassifier(learning_rate = 0.05, n_estimators=300, max_depth=5)
XG.fit(X_train, y_train)
xgb_pred = XG.predict(X_test)

In [9]:
from sklearn.metrics import f1_score

f1_score(y_test,xgb_pred,average='macro')

0.6498106347523103

In [13]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_pred = svm_model_linear.predict(X_test) 
f1_score(y_test,svm_pred,average='mcro')

0.44434450081508914

In [14]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 5, weights='distance').fit(X_train, y_train) 
knn.predict(X_test)  


0.6624508611955421

In [15]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e50, solver='liblinear').fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)  
f1_score(y_test,logreg_pred,average='macro')

C:\Users\Rosiel Jazmine V\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.39142645971914264